In [1]:
import torch
print(torch.cuda.is_available())

ModuleNotFoundError: No module named 'torch'

In [6]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [41]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


In [42]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
# import sys
# from PIL import Image
# sys.modules['Image'] = Image 

In [47]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam

# Define directories
train_dir = "/Users/ayushjain/Downloads/NEW_PLANT_DATASET/TRAIN"
val_dir = "/Users/ayushjain/Downloads/NEW_PLANT_DATASET/VALID"
test_dir = "/Users/ayushjain/Downloads/NEW_PLANT_DATASET/TEST"

# Define data generators for training and validation with augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Define batch size
batch_size = 8

# Create generators to load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='sparse'  # Adjust class_mode based on your classification task
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='sparse'  # Adjust class_mode based on your classification task
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False  # Adjust class_mode based on your classification task
)

# Load pre-trained InceptionV3 model
inception = InceptionV3(input_shape=(299, 299, 3), weights='imagenet', include_top=False)

# Freeze pre-trained layers
for layer in inception.layers:
    layer.trainable = False

# Add custom dense layers for classification
x = Flatten()(inception.output)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(10, activation='softmax')(x)

# Create the model
model = Model(inputs=inception.input, outputs=prediction)

# Compile the model
optimizer = Adam(lr=0.001)  # Initial learning rate
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)



Found 8418 images belonging to 10 classes.
Found 2173 images belonging to 10 classes.
Found 3634 images belonging to 10 classes.


In [48]:
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_1316 (Conv2D)        (None, 149, 149, 32)         864       ['input_15[0][0]']            
                                                                                                  
 batch_normalization_1316 (  (None, 149, 149, 32)         96        ['conv2d_1316[0][0]']         
 BatchNormalization)                                                                              
                                                                                                  
 activation_1316 (Activatio  (None, 149, 149, 32)         0         ['batch_normalization_1

In [45]:
from PIL import Image

In [50]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,  # Adjust the number of epochs
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[early_stop, reduce_lr]
)

# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('inceptionV1.png')
plt.show()



Epoch 1/50


KeyboardInterrupt: 

In [ ]:
# Testing the model
test_steps = test_generator.n / batch_size
test_loss, test_accuracy = model.evaluate_generator(test_generator, steps=test_steps, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Generating predictions for the test set
test_generator.reset()  # Reset generator before calling predict_generator
predictions = model.predict_generator(test_generator, steps=test_steps, verbose=1)
predicted_classes = predictions.argmax(axis=-1)

# Creating the confusion matrix
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

cm = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(cm)

# Plotting confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Adding text annotations
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

# Classification report
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))


/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/PIL/Image.py


In [ ]:
tf.keras.models.load_model('vgg.h5')

/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/PIL/Image.py


In [40]:
def load_image(img_path, show=False):
        img = image.load_img(img_path, target_size=(299,299))
        img_tensor = np.expand_dims(img, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
        img_tensor = preprocess_input(img_tensor)
        
        if show:
            plt.imshow(img_tensor[0])
            plt.axis('off')
            plt.show()
        
        return img_tensor

load_image('/Users/ayushjain/Downloads/NEW_PLANT_DATASET/VALID/Lemon/0110.jpg', True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ayushjain/Downloads/NEW_PLANT_DATASET/VALID/Lemon/0110.jpg'